In [130]:
import pandas as pd
from io import StringIO
import os
import re
import numpy as np
import subprocess

In [2]:
problem_entries_path = 'problem_entries.txt'
problem_entries=open(problem_entries_path,"r").read().split('\n')

In [3]:
problem_entries

['calum', 'jamesc', 'behzad', 's0343804', 's0346429', 's0451815', 's0455730']

In [4]:
server_path = '/Volumes/Network/courses/sp/data/'

In [5]:
data_path = 'info.txt'
data_file = open(os.path.join(server_path,data_path),"r").read().split('\n')

In [6]:
data_file

['Information about the data. Only isolated English digits data are listed here.',
 '',
 '',
 'Microphone types:',
 '',
 'two types where the micophone could be at a variable distance from the speaker:',
 '"big"    : stand-mounted Sennheiser mikes (good quality)',
 '"small"  : basic Sun lapel type mic (low quality)',
 '',
 'plus various types of headset microphone:',
 '"headset" (medium quality)',
 '"logitech_headset"',
 '"sennheiser_headset"',
 '',
 'plus a variety of other microphone types, described later in this file.',
 '',
 '',
 'Where two microphone names appear (e.g. "headset/big") they are for',
 'train/test data respectively',
 '',
 'Accent types:',
 'SC : native Scottish English speaker',
 'UK : native other British English speaker',
 'AZ : native Australian/NZ speaker',
 'NA : native North American English speaker',
 'NN : non-native',
 '',
 '',
 '=================================================================',
 '',
 'There are known to be some problems with files from t

In [7]:
year_pattern = re.compile("Year*")

In [8]:
entry_pattern = re.compile("[a-z]")

In [9]:
line_pattern = re.compile('[\. a-zA-z]*:+')

In [10]:
data_dict = {}
for line in data_file:
    if year_pattern.match(line):
        year = line.split(" ")[1]
        continue
    if (entry_pattern.match(line) and (not line_pattern.match(line))):
        line = re.split(" +|\t+ *",line)
        if (len(line)>3 and len(line)<7):
            line[3:] = [' '.join(line[3:6])]
            data_dict.setdefault(year,[]).append(line)

In [11]:
data_dict.keys()

dict_keys(['2000-1', '2001-2', '2002-3', '2003-4', '2004-5', '2005-6', '2006-7', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20'])

In [12]:
tuples = []
for key in data_dict.keys():
    entry_num = len(data_dict[key])
    key_list = [key]*entry_num
    index_list = range(entry_num)
    tuples = tuples+(list(zip(key_list,index_list)))

In [13]:
index = pd.MultiIndex.from_tuples(tuples, names=['Year','Index'])

In [14]:
info_df = pd.DataFrame(columns=['Name','Gender','Microphone','Accent'])
for key in data_dict.keys():
    year_df = pd.DataFrame(data_dict[key],columns=['Name','Gender','Microphone','Accent'])
    info_df=info_df.append(year_df)

In [15]:
info_df.index=index

In [16]:
info_df.shape

(493, 4)

In [17]:
gender = info_df.loc['2013-14']['Microphone'].copy(True)
mic = info_df.loc['2013-14']['Accent'].copy(True)
accent = info_df.loc['2013-14']['Gender'].copy(True)
info_df.loc['2013-14']['Microphone']=mic
info_df.loc['2013-14']['Gender']=gender
info_df.loc['2013-14']['Accent']=accent

In [18]:
info_df.loc['2013-14']

,Name,Gender,Microphone,Accent
Index,,,,
0,s1312082,Male,logitech_headset Praat Lab,NN
1,s1355318,Female,logitech_headset Audacity Lab,NN
2,s1367766,Female,logitech_headset Praat Lab,NN
3,s1038946,Male,Apple_EarPods Audacity Lab,NN
4,s0950147,Female,iMac_inbuilt Audacity Lab,UK
5,s1336117,Female,iMac_inbuilt Audacity Lab,NN
6,s1036740,Female,logitech_headset Praat Lab,NN
7,s1114083,Female,Acer_v5-571_inbuilt Audacity Home,NA
8,s0927462,Female,iMac_inbuilt Audacity Lab,UK


In [19]:
for p in problem_entries:
    indexName = info_df[info_df['Name']==p].index
    info_df.drop(indexName,inplace=True)

In [20]:
info_df.to_csv('recording_info.csv')

## Specify the selecting requirements

In [21]:
selection = info_df[~info_df['Gender'].isin(['f','Female'])]['Name'].values

In [22]:
selection

array(['chongh', 'jamesh', 'markm', 'scottn', 'simonk', 'alex',
       'anthonyb', 'benjamin', 'ccb', 'christop', 'colinb', 'davidr',
       'yokong', 'athan', 'davidt', 'guy', 'jose', 'michaelg', 'michaels',
       'ptepper', 'spengler', 'timm', 'wouter', 's0345201', 's9901729',
       's0345701', 's0340933', 's0348129', 's0344328', 's0343559',
       's0345347', 's0343444', 'joshua', 's0343879', 's0343799',
       's0345336', 's0349492', 's9810638', 's0126339', 's0450736',
       's0453481', 's0454291', 's0455511', 's0457086', 'taal11',
       's0235479', 's0565563', 's0451342', 's0672048', 's0677387',
       's0678004', 's0680940', 's0681481', 's0836504', 's1052161',
       's1052483', 's1054017', 's9061219', 's0680570', 's0806577',
       's0824968', 's0832212', 's0833088', 's0927722', 's0940658',
       's1100218', 's1125835', 's1128289', 's1135029', 's1151467',
       's1157551', 's0906559', 's0907414', 's0949139', 's1021894',
       's1200359', 's1250520', 's1261038', 's1312082'

Generate the files for HTK

In [23]:
# Function for writing the List_inti_train list
# `names` is a list of user names selected before
# `path` is the path to the scp file that we want to write to
def prepareInit(names,path):
    print("Preparing training data list...\n")
    with open(path,'w+') as init:
        for name in names:
            print(name)
            init.write("/Volumes/Network/courses/sp/data_upload/mfcc/train/{}_train.mfcc\n".format(name))
    init.close()
    print("Done\n")
def prepareRecog(names,path):
    print("Preparing testing data list...\n")
    with open(path,'w+') as rec:
        for name in names:
            print(name)
            rec.write("/Volumes/Network/courses/sp/data_upload/mfcc/test/{}_test*.mfcc\n".format(name))
    rec.close()
    print("Done\n")
def prepare_res_label(names,path):
    print("Preparing testing master files list...\n")
    with open(path,'w+') as lab:
        for name in names:
            lab.write("/Volumes/Network/courses/sp/data_upload/lab/test/{}_test.mlf\n".format(name))
            print(name)
    lab.close()
    print("Done\n")
def prepare_res_rec(names,recogresult_path,list_path):
    print("Preparing testing recognition results...\n")
    file_names = [rec for rec in os.listdir(recogresult_path) if rec.split('_')[0] in names]
    with open(list_path,'w+') as rec_result:
        for file_name in file_names:
            rec_result.write("../rec/{}\n".format(file_name))
            print(file_name)
    rec_result.close()
    print("Done\n")

In [24]:
# The path for all lists and specific lists
# Path to the directory containing all the recognition results
recogresult = "../rec/"
# Path to the lists for traininig and testing
listpath = "../list/"
initpath = os.path.join(listpath,"list_init_train.scp")
recopath = os.path.join(listpath,"list_recogniser.scp")
labepath = os.path.join(listpath,"list_results_label.scp")
r_repath = os.path.join(listpath,"list_results_rec.scp")

In [25]:
initpath

'../list/list_init_train.scp'

## Experiment 1 | test with a five users' data

In [116]:
selection = info_df.loc['2018-19'].head();

In [117]:
selection=selection['Name'].values;selection

array(['s1777759', 's1881336', 's1834065', 's1873815', 's1870525'],
      dtype=object)

In [168]:
def run_experiment(trn_list,tst_list):
    prepareInit(trn_list,initpath)
    prepareRecog(trn_list,recopath)
    subprocess.call('../scripts/mmulti_init_train')
    prepare_res_label(trn_list,labepath)
    prepare_res_rec(trn_list,recogresult,r_repath)
    try:
        result=subprocess.check_output('../scripts/mmulti_recognise_result')
        print(result)
    except:
        print("bad user")

In [170]:
run_experiment(['simonk'],['simonk'])

Preparing training data list...

simonk
Done

Preparing testing data list...

simonk
Done

Preparing testing master files list...

simonk
Done

Preparing testing recognition results...

simonk_test13.rec
simonk_test16.rec
simonk_test09.rec
simonk_test19.rec
simonk_test25.rec
simonk_test10.rec
simonk_test06.rec
simonk_test12.rec
simonk_test01.rec
simonk_test26.rec
simonk_test29.rec
simonk_test27.rec
simonk_test17.rec
simonk_test11.rec
simonk_test21.rec
simonk_test23.rec
simonk_test20.rec
simonk_test14.rec
simonk_test30.rec
simonk_test28.rec
simonk_test03.rec
simonk_test04.rec
simonk_test05.rec
simonk_test18.rec
simonk_test24.rec
simonk_test15.rec
simonk_test22.rec
simonk_test08.rec
simonk_test07.rec
simonk_test02.rec
Done

b'WORD: %Corr=73.33, Acc=73.33 [H=22, D=0, S=8, I=0, N=30]\n'


In [157]:
! ../scripts/mmulti_recognise_result

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/nicolek_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Config parameters invalid
 FATAL ERROR - Terminating program HVite
  ERROR [+6510]  LoadMasterFile: cannot open MLF /Volumes/Network/courses/sp/data_upload/lab/test/nicolek_test.mlf
 FATAL ERROR - Terminating program HResults


## DONE

## Experient 2 | Remove users with WER higher than 10%

In [148]:
all_users = info_df['Name'].values;all_users

array(['nicolek', 'chongh', 'jamesh', 'markm', 'scottn', 'simonk',
       'annika', 'beatrice', 'elisabet', 'fionac', 'michele', 'nicola',
       'panagiot', 'sarah', 'sasha', 'alex', 'anthonyb', 'benjamin',
       'ccb', 'christop', 'colinb', 'davidr', 'yokong', 'andra', 'angie',
       'beata', 'efi', 'kateb', 'kelly', 'kit', 'kristina', 'mariza',
       'milena', 'rachel', 'stav', 'unni', 'vwiersma', 'yoko', 'athan',
       'davidt', 'guy', 'jose', 'michaelg', 'michaels', 'ptepper',
       'spengler', 'timm', 'wouter', 's0345201', 's0343373', 's9901729',
       's0345701', 's0345310', 's0340933', 's0341098', 's0348129',
       's0343949', 's0344328', 's0343559', 's0345347', 's0343820',
       's0343444', 'joshua', 's0343879', 's0343799', 's0348176',
       's0349395', 's0345336', 's0349492', 's0339733', 's9810638',
       's0340681', 's0125605', 's0126339', 's0127196', 's0231469',
       's0450736', 's0450918', 's0450928', 's0450990', 's0453481',
       's0453738', 's0453767', 's045

In [99]:
bad_users = []
for user in all_users:
    result = run_experiment([user],[user])
    
    # Check if the user is of good quality and return the %Corr 
#     check_user(user)
#     if score < 100:
#         bad_users.append(user)
bad_users

Preparing training data list...

n
i
c
o
l
e
k
Done

Preparing testing data list...

n
i
c
o
l
e
k
Done

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/train/n_train.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+2150]  LoadFile: Config parameters invalid
 FATAL ERROR - Terminating program HInit
  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/train/n_train.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+2150]  LoadFile: Config parameters invalid
 FATAL ERROR - Terminating program HInit
  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/train/n_train.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+2150]  LoadFile: Confi

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/e_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Config parameters invalid
 FATAL ERROR - Terminating program HVite
  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/k_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Config parameters invalid
 FATAL ERROR - Terminating program HVite
  ERROR [+6510]  LoadMasterFile: cannot open MLF /Volumes/Network/courses/sp/data_upload/lab/test/n_test.mlf
 FATAL ERROR - Terminating program HResults
Preparing training data list...

c
h
o
n
g
h
Done

Preparing testing data list...

c
h
o
n
g
h
Done

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/tra

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/h_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Config parameters invalid
 FATAL ERROR - Terminating program HVite
  ERROR [+6510]  LoadMasterFile: cannot open MLF /Volumes/Network/courses/sp/data_upload/lab/test/c_test.mlf
 FATAL ERROR - Terminating program HResults
Preparing training data list...

j
a
m
e
s
h
Done

Preparing testing data list...

j
a
m
e
s
h
Done

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/train/j_train.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+2150]  LoadFile: Config parameters invalid
 FATAL ERROR - Terminating program HInit
  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/train

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/s_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Config parameters invalid
 FATAL ERROR - Terminating program HVite
  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/h_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Config parameters invalid
 FATAL ERROR - Terminating program HVite
  ERROR [+6510]  LoadMasterFile: cannot open MLF /Volumes/Network/courses/sp/data_upload/lab/test/j_test.mlf
 FATAL ERROR - Terminating program HResults
Preparing training data list...

m
a
r
k
m
Done

Preparing testing data list...

m
a
r
k
m
Done

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/train/m

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/k_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Config parameters invalid
 FATAL ERROR - Terminating program HVite
  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/m_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Config parameters invalid
 FATAL ERROR - Terminating program HVite
  ERROR [+6510]  LoadMasterFile: cannot open MLF /Volumes/Network/courses/sp/data_upload/lab/test/m_test.mlf
 FATAL ERROR - Terminating program HResults
Preparing training data list...

s
c
o
t
t
n
Done

Preparing testing data list...

s
c
o
t
t
n
Done

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/tra

...

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/train/s_train.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+2250]  LoadFile: Config parameters invalid
 FATAL ERROR - Terminating program HRest
  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/train/s_train.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+2250]  LoadFile: Config parameters invalid
 FATAL ERROR - Terminating program HRest
  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/train/s_train.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+2250]  LoadFile: Config parameters invalid
 FATAL ERROR - Terminating program HRest
  ERROR [+6310]  OpenParmChannel: cannot op

Preparing testing master files list...

u
n
n
i
Done

Preparing testing recognition results...

Done

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/u_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Config parameters invalid
 FATAL ERROR - Terminating program HVite
  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/n_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Config parameters invalid
 FATAL ERROR - Terminating program HVite
  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/n_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Co

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/i_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Config parameters invalid
 FATAL ERROR - Terminating program HVite
  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/e_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Config parameters invalid
 FATAL ERROR - Terminating program HVite
  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/r_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Config parameters invalid
 FATAL ERROR - Terminating program HVite
  ERROR [+6310]  OpenParmChannel: can

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/train/y_train.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+2250]  LoadFile: Config parameters invalid
 FATAL ERROR - Terminating program HRest
Preparing testing master files list...

y
o
k
o
Done

Preparing testing recognition results...

Done

  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/y_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Config parameters invalid
 FATAL ERROR - Terminating program HVite
  ERROR [+6310]  OpenParmChannel: cannot open Parm File /Volumes/Network/courses/sp/data_upload/mfcc/test/o_test*.mfcc
  ERROR [+6313]  OpenAsChannel: OpenParmChannel failed
  ERROR [+6316]  OpenBuffer: OpenAsChannel failed
  ERROR [+3250]  ProcessFile: Conf

KeyboardInterrupt: 